In [1]:
def outer():
    # closure begin
    x = 'python'

    def inner():
        print(f"{x} rocks!")
    # end closure
    inner()

outer()

python rocks!


when we return **inner**, we are actually "returning" the **closure**

In [ ]:
def outer():
    # closure begin
    x = 'python'

    def inner():
        print(f"{x} rocks!")
    # end closure
    return inner


we can assign the return value to a variable name: **fn = outer()** -> **fn()**

When we called fn at that time Python determined the value of x in the extended scope.

But notice that **outer** has finished running **before** we call **fn** - it's scope was "gone"

# Python Cells and Multi-Scoped Variables

Here the value of x is **shared** between 2 scopes:
- outer
- closure

The label x is in 2 different scopes but always reference the same "value"

Python does this by creating a **cell** as an intermediary object:

outer.x --> cell
              |    ---> str ('python')
inner.x --> cell

both x (in outer and inner) point to the **same** cell

# Closures

Closures: **a function + extended scope** that contains **free variables**

In [4]:
def outer():
    a = 100
    # closure begin
    x = 'python'

    def inner():
        a = 10 # local variable
        print(f"{x} rocks!")
    # end closure
    return inner

fn = outer()
print(fn.__code__.co_freevars) # -> ('x',)
print(fn.__closure__)  # -> tuple of cell object (<cell at 0x...: str object at 0x..., )

('x',)
(<cell at 0x000001AE559832B0: str object at 0x000001AE5387BAB0>,)


In [5]:
def outer():
    # closure begin
    x = 'python'
    print(hex(id(x))) # indirect reference
    def inner():
        print(hex(id(x)))
        print(f"{x} rocks!")
    # end closure
    return inner

fn = outer()
fn()

0x1ae5387bab0
0x1ae5387bab0
python rocks!


# Modifying free variables

In [6]:
def counter():
    count = 0
    def inc():
        nonlocal count
        count += 1
        return count
    return inc

fn = counter()
fn() # -> 1 count indirect reference changed from object 0 to object 1
fn()

2

# Multiple instances of Closures

Every time we run a function, a **new** scope is created

If that function generates a closure, a **new** closure is created every time as well

In [11]:
def counter():
    count = 0

    def inc():
        nonlocal count
        count += 1
        return count
    return inc

f1 = counter()
f2 = counter()

print(f1.__closure__) # f1 and f2 do not have the same extended scope, they are different instances of closure -> the cell are different
print(f2.__closure__)

(<cell at 0x000001AE55983880: int object at 0x000001AE509F6910>,)
(<cell at 0x000001AE55983F40: int object at 0x000001AE509F6910>,)


# Shared extended scopes

In [12]:
def counter():
    count = 0

    def inc1():
        nonlocal count
        count += 1
        return count

    def inc2():
        nonlocal count
        count += 1
        return count

    return inc1, inc2

f1, f2 = counter()
f1() # -> 1
f2() # -> 2

2

**count** is a free variable - bound to **count** in the extended scope

You may think this shared extended scope is highly unusual... but it's not!

In [13]:
# No shared extended scopes

def adder(n):
    def inner(x):
        return x + n
    return inner

add_1 = adder(1)
add_2 = adder(2)
add_3 = adder(3)

print(add_1(10))
print(add_2(10))
print(add_3(10))

11
12
13


In [16]:
adders = []
for n in range(1, 4):
    adders.append(lambda x: x + n)

# n is global variable, not free variable
# adder is not closure

print(adders[0](10))
print(adders[1](10))
print(adders[2](10))

print(adders[0].__code__.co_freevars)
print(adders[0].__closure__)

13
13
13
()
None


In [15]:
# Nested Closures

def increment(n):
    # inner + n is a closure
    def inner(start):
        current = start
        # inc + current + n is a closure
        def inc():
            nonlocal current
            current += n
            return current
        return inc
    return inner

fn = increment(2)
print(fn.__code__.co_freevars)

inc_2 = fn(100)
print(inc_2.__code__.co_freevars)

print(inc_2())
print(inc_2())

('n',)
('current', 'n')
102
104


# Closures Application

In [17]:
class Averager:
    def __init__(self):
        self.numbers = []

    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count

a = Averager()
a.add(10)
a.add(20)
a.add(30)

20.0

In [18]:
# Closure

def averager():
    numbers = []
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total / count
    return add

a = averager()
a(10)


10.0

In [19]:
a(20)

15.0

In [20]:
a(30)

20.0

In [21]:
def averager1():
    total = count = 0
    def add(number):
        nonlocal total
        nonlocal count
        total += number
        count += 1
        return total / count
    return add

a = averager1()
print(a.__closure__)
print(a.__code__.co_freevars)

(<cell at 0x000001AE5596D700: int object at 0x000001AE509F6910>, <cell at 0x000001AE5596D6D0: int object at 0x000001AE509F6910>)
('count', 'total')


In [22]:
a(10)

10.0

In [23]:
a(20)

15.0

In [ ]:
# class version
class Averager:
    def __init__(self):
        self.total = 0
        self.count = 0

    def add(self, number):
        total += number
        count += 1
        return total / count

In [27]:
from time import perf_counter

class Timer:
    def __init__(self) -> None:
        self.start = perf_counter()

    def __call__(self):
        return perf_counter() - self.start

t1 = Timer()

In [28]:
t1()

9.121120900003007

In [29]:
t1()

12.867424500000197

In [30]:
def timer():
    start = perf_counter()

    def poll():
        return perf_counter() - start

    return poll

t2 = timer()

In [31]:
t2()

5.741308299999218

In [33]:
t2()

14.400963699998101

In [37]:
def counter(initial_value=0):
    def inc(increment=1):
        nonlocal initial_value
        initial_value += increment
        return initial_value
    return inc

c_1 = counter()

In [38]:
c_1()

1

In [39]:
c_1()

2

In [40]:
def counter(fn):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        print(f"{fn.__name__} has been called {cnt} times")
        return fn(*args, **kwargs)
    return inner

In [41]:
def add(a, b):
    return a + b

def mul(a, b):
    return a * b

In [42]:
counter_add = counter(add)

counter_add.__closure__

(<cell at 0x000001AE55AF00D0: int object at 0x000001AE509F6910>,
 <cell at 0x000001AE55AF04F0: function object at 0x000001AE56D8DCA0>)

In [43]:
counter_add.__code__.co_freevars

('cnt', 'fn')

In [46]:
hex(id(add))

'0x1ae56d8dca0'

In [47]:
result = counter_add(10, 20)

add has been called 1 times


In [48]:
counter_add(11, 20)

add has been called 2 times


31

In [49]:
def counter_dict(fn, counters):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        counters[fn.__name__] = cnt
        return fn(*args, **kwargs)
    return inner

c = dict()
counted_add = counter_dict(add, c)
counted_mul = counter_dict(mul, c)

In [50]:
counted_add(1, 2)
counted_add(23, 30)
counted_add(28, 30)
counted_mul(4,5)
c

{'add': 3, 'mul': 1}